In [ ]:
import os
os.chdir('/home/berling/reduce_opto_response/')

# global variabl setup
neuron_model_name = "L23_PC_cADpyr229_1"
adex_fit = 'snake_workflow/adex_models/fit_to_L23_PC_cADpyr229_1.nestml'
ChR_soma_density=13e9
ChR_distribution='uniform'
stimulator_dict = dict(
        diameter_um=50,
        NA=0.1)
temp_protocol=dict(
    duration_ms=200,
    delay_ms=1,
    total_rec_time_ms=300,
    interpol_dt_ms=1
)
interpol_dt_ms = temp_protocol['interpol_dt_ms']

stim_intensity_mWPERmm1 = 0.2

driving_conductance_nS = np.load()

In [ ]:
# Run full simulation
from neurostim.analysis import quick_sim_setup
from neurostim.cell import Cell
from neurostim import models
import numpy as np
import pandas as pd
from neuron import h
from scipy.signal import find_peaks

simcontrol = quick_sim_setup(
    cell_dict = dict(
        cellmodel=neuron_model_name,
        ChR_soma_density=ChR_soma_density,
        ChR_distribution=ChR_distribution),
    stimulator_dict = stimulator_dict
)

# Define recording variables
rec_vars = [[],[]]
# append time and soma voltage recoding
rec_vars[0].append('time [ms]')
rec_vars[1].append(h._ref_t)
rec_vars[0].append('v_soma_mV')
rec_vars[1].append(simcontrol.cell.model.soma_sec(0.5)._ref_v)

tmp = simcontrol.run(
    temp_protocol=temp_protocol,
    stim_location=(0, 0, 0),
    stim_intensity_mWPERmm2=stim_intensity_mWPERmm2,
    rec_vars=rec_vars,
    interpol_dt_ms=interpol_dt_ms
)
# measure APC
v_soma = tmp[['time [ms]', 'v_soma_mV']]
v_soma_until_stim_period_stops = v_soma.loc[v_soma['time [ms]']<=temp_protocol['duration_ms']+temp_protocol['delay_ms']]
x = tmp['time [ms]']
y = tmp['v_soma_mV']
peaks, properties = find_peaks(y, height=-20, prominence=10)
np.save('spike_times_full.npy', peaks)

In [ ]:
# Run somatic-equivalent conductance at soma of HH-model
from neurostim.analysis import quick_sim_setup
from neurostim.cell import Cell
from neurostim import models
import numpy as np
import pandas as pd
from neuron import h
from scipy.signal import find_peaks

time_ms = np.arange(0,temp_protocol['total_rec_time_ms'],temp_protocol['interpol_dt_ms'])
# set up neuron model
cellmodel = getattr(models, neuron_model_name)
cell = Cell(
    model=cellmodel(),
    ChR_soma_density=13e9,
    ChR_distribution='uniform'
)
# insert new conductance to 'inject' conductance
soma = cell.sections[0]
soma.insert('g_chanrhod')

# define/load driving stimulus
conductance_nS = np.load(str(snakemake.input[0]))[0]

# driving stimulus
t = h.Vector(time_ms)
y = h.Vector(conductance_nS*(100/soma(0.5).area())) # needs to be rescaled as gcat is density mechanism

# Define recording variables
rec_vars = [[],[]]
rec_time = h.Vector().record(h._ref_t)
rec_v = h.Vector().record(soma(0.5)._ref_v)

# run simulation with injected conductance
h.load_file('stdrun.hoc')

# play the stimulus into soma(0.5)'s ina
# the last True means to interpolate; it's not the default, but unless
# you know what you're doing, you probably want to pass True there
y.play(soma(0.5)._ref_gcat2_g_chanrhod, t, True)

# h.v_init, h.tstop= -70, temp_protocol['total_rec_time_ms']
h.v_init, h.tstop= -70, 201
h.run()

# measure APC
x = np.array(rec_time)
y = np.array(rec_v)
peaks, properties = find_peaks(y, height=-20, prominence=10)
np.save('spike_times_RON_full.npy', peaks)

In [ ]:
# Run somatic-equivalent conductance at AdEx - use nest env
import numpy as np
import pandas as pd
import nest
from pynestml.codegeneration.nest_code_generator_utils import NESTCodeGeneratorUtils

module_name, neuron_model_name = NESTCodeGeneratorUtils.generate_code_for(adex_fit)

def nest_sim(stim_times, stim_cond):
    #nest.set_verbosity("M_WARNING")
    #nest.set_verbosity("M_ERROR")
    nest.ResetKernel()

    nest.Install(module_name)

    neuron = nest.Create(neuron_model_name)

    voltmeter = nest.Create("voltmeter")
    voltmeter.set({"record_from": ["V_m"]})
    nest.Connect(voltmeter, neuron)

    sr = nest.Create("spike_recorder")
    nest.Connect(neuron, sr)

    scg = nest.Create('step_current_generator')

    nest.Connect(scg, neuron)
    
    scg.set({'amplitude_times':stim_times, 'amplitude_values':stim_cond,'stop':stim_times[-1]})

    nest.Simulate(stim_times[-1])

    return voltmeter.get("events")["times"], voltmeter.get("events")["V_m"], nest.GetStatus(sr, keys='events')[0]['times']

time_ms = np.arange(0,temp_protocol['total_rec_time_ms'],temp_protocol['interpol_dt_ms'])
# define/load driving stimulus
conductance_nS = driving_conductance_nS

# take time 0 away (nest does not like time 0):
time_ms = time_ms[1:]
conductance_nS = conductance_nS[1:]
# convert times to float as nest expects float not int
time_ms = time_ms.astype(float)

times, Vm, spike_times = nest_sim(time_ms, conductance_nS)

np.save('spike_times_RON_adex.npy', spike_times)